In [0]:
!git clone https://github.com/google-research/bert.git

fatal: destination path 'bert' already exists and is not an empty directory.


In [0]:
ls -l

total 8
drwxr-xr-x 3 root root 4096 Apr 11 05:25 bert/
drwxr-xr-x 1 root root 4096 Apr  3 16:24 sample_data/


In [0]:
cd bert

/gdrive/My Drive/bert


In [0]:
ls -l

total 400
-rw-r--r-- 1 root root  1323 Apr 11 05:25 CONTRIBUTING.md
-rw-r--r-- 1 root root 16475 Apr 11 05:25 create_pretraining_data.py
-rw-r--r-- 1 root root 13898 Apr 11 05:25 extract_features.py
-rw-r--r-- 1 root root   616 Apr 11 05:25 __init__.py
-rw-r--r-- 1 root root 11358 Apr 11 05:25 LICENSE
-rw-r--r-- 1 root root 37922 Apr 11 05:25 modeling.py
-rw-r--r-- 1 root root  9191 Apr 11 05:25 modeling_test.py
-rw-r--r-- 1 root root 11242 Apr 11 05:25 multilingual.md
-rw-r--r-- 1 root root  6258 Apr 11 05:25 optimization.py
-rw-r--r-- 1 root root  1721 Apr 11 05:25 optimization_test.py
-rw-r--r-- 1 root root 66488 Apr 11 05:25 predicting_movie_reviews_with_bert_on_tf_hub.ipynb
-rw-r--r-- 1 root root 50519 Apr 11 05:25 README.md
-rw-r--r-- 1 root root   110 Apr 11 05:25 requirements.txt
-rw-r--r-- 1 root root 34783 Apr 11 05:25 run_classifier.py
-rw-r--r-- 1 root root 11426 Apr 11 05:25 run_classifier_with_tfhub.py
-rw-r--r-- 1 root root 18667 Apr 11 05:25 run_pretraining.py
-rw-r--r-

In [0]:
sys.path.append('biobert_v1.0_pubmed_pmc')
!gsutil mv biobert_v1.0_pubmed_pmc $BUCKET_NAME

Copying file://biobert_v1.0_pubmed_pmc/bert_config.json [Content-Type=application/json]...
Removing file://biobert_v1.0_pubmed_pmc/bert_config.json...
Copying file://biobert_v1.0_pubmed_pmc/biobert_model.ckpt.index [Content-Type=application/octet-stream]...
Removing file://biobert_v1.0_pubmed_pmc/biobert_model.ckpt.index...
Copying file://biobert_v1.0_pubmed_pmc/biobert_model.ckpt.meta [Content-Type=application/octet-stream]...
Removing file://biobert_v1.0_pubmed_pmc/biobert_model.ckpt.meta...
Copying file://biobert_v1.0_pubmed_pmc/vocab.txt [Content-Type=text/plain]...
Removing file://biobert_v1.0_pubmed_pmc/vocab.txt...

==> NOTE: You are performing a sequence of gsutil operations that may
run significantly faster if you instead use gsutil -m cp ... Please
see the -m section under "gsutil help options" for further information
about when gsutil -m can be advantageous.

Copying file://biobert_v1.0_pubmed_pmc/biobert_model.ckpt.data-00000-of-00001 [Content-Type=application/octet-stream]

In [1]:
from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive/'My Drive'

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /gdrive
/gdrive/My Drive


In [2]:
import sys
sys.path.append('Data')
%tensorflow_version 1.x

TensorFlow 1.x selected.


In [0]:
ls -l

total 34641
-rw------- 1 root root    69561 Apr 11 20:57  1R84voFKHfWV9xjzeLzWBbmY1uOMYpnyD
drwx------ 2 root root     4096 Oct  3  2015  201551013/
-rw------- 1 root root  1058420 Apr  7  2017  201551013_assignment4.pdf
drwx------ 2 root root     4096 Jan  2  2016 '201551013@iiitvadodara.ac.in'/
-rw------- 1 root root   347350 Dec  2  2017  201551013_IIITV.pdf
-rw------- 1 root root   852484 Aug  1  2019  201551013.pdf
-rw------- 1 root root   895505 Apr  7  2017  201551033_assignment4.pdf
-rw------- 1 root root   642244 Feb 19  2017  A17102657142.pdf
-rw------- 1 root root      144 Feb 24  2017 'Aditya Krishna Sai.vcf'
-rw------- 1 root root   222390 Jun 11  2017  AdityaSai_IndianInstituteOfTechnologyVadodara_ID.pdf
-r-------- 1 root root      151 May 20  2019 'Amazon Prime CTK TOONS.gdoc'
-rw------- 1 root root  2029387 Feb 15  2018 "Are MNC 's devils in disguise.pdf"
-rw------- 1 root root      151 Apr  8 06:13 'Assignment 3.gdoc'
drwx------ 2 root root     4096 Apr 11 05:24  bert/

In [3]:
import datetime
import json
import os
import pprint
import random
import string
import sys
import tensorflow as tf

assert 'COLAB_TPU_ADDR' in os.environ, 'ERROR: Not connected to a TPU runtime; please see the first cell in this notebook for instructions!'
TPU_ADDRESS = 'grpc://' + os.environ['COLAB_TPU_ADDR']
print('TPU address is => ', TPU_ADDRESS)

from google.colab import auth
auth.authenticate_user()
with tf.Session(TPU_ADDRESS) as session:
  print('TPU devices:')
  pprint.pprint(session.list_devices())

  # Upload credentials to TPU.
  with open('/content/adc.json', 'r') as f:
    auth_info = json.load(f)
  tf.contrib.cloud.configure_gcs(session, credentials=auth_info)
  # Now credentials are set for all future sessions on this TPU.

TPU address is =>  grpc://10.81.103.130:8470
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

TPU devices:
[_DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:CPU:0, CPU, -1, 8995465402990602291),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 17179869184, 8478597538903208990),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:0, TPU, 17179869184, 5082715199462448318),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:1, TPU, 17179869184, 14943111799037024054),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:2, TPU, 17179869184, 14360733434453511544),
 _DeviceAttributes(/job:tpu_worker/replica:0/ta

In [4]:
BUCKET = 'cse576-nlp-project' #@param {type:"string"}
assert BUCKET, '*** Must specify an existing GCS bucket name ***'
output_dir_name = 'output_biobertlarge_summarized_1.0' #@param {type:"string"}
BUCKET_NAME = 'gs://{}'.format(BUCKET)
OUTPUT_DIR = 'gs://{}/{}'.format(BUCKET,output_dir_name)
tf.gfile.MakeDirs(OUTPUT_DIR)
print('***** Model output directory: {} *****'.format(OUTPUT_DIR))

***** Model output directory: gs://cse576-nlp-project/output_biobertlarge_summarized_1.0 *****


In [5]:
! $OUTPUT_DIR

/bin/bash: gs://cse576-nlp-project/output_biobertlarge_summarized_1.0: No such file or directory


In [6]:
!pip install sentencepiece
!pip install tokenizers
!pip install sacremoses


     |████████████████████████████████| 1.0MB 3.4MB/s 
     |████████████████████████████████| 3.8MB 3.4MB/s 
     |████████████████████████████████| 890kB 3.5MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.41-cp36-none-any.whl size=893334 sha256=2160b726ca7105b34a273b872d8c77d989fdb262dc897f108fb4a6287b7d4075
  Stored in directory: /root/.cache/pip/wheels/22/5a/d4/b020a81249de7dc63758a34222feaa668dbe8ebfe9170cc9b1
Successfully built sacremoses


Command to run the model. Train file and test file arguments takes the train and test data sets respectively. Output directory contains the train and test models, checkpoint files and predictions files 

In [0]:
!python bert/run_squad.py \
  --vocab_file=$BUCKET_NAME/biobert_large/vocab_cased_pubmed_pmc_30k.txt \
  --bert_config_file=$BUCKET_NAME/biobert_large/bert_config_bio_58k_large.json \
  --init_checkpoint=$BUCKET_NAME/biobert_large/bio_bert_large_1000k.ckpt \
  --do_train=True \
  --train_file=Summarizedv1/bert_train.json \
  --do_predict=True \
  --predict_file=Summarizedv1/bert_dev.json \
  --train_batch_size=24 \
  --learning_rate=3e-5 \
  --num_train_epochs=2.0 \
  --use_tpu=True \
  --tpu_name=grpc://10.47.230.114:8470 \
  --max_seq_length=384 \
  --doc_stride=128 \
  --version_2_with_negative=True \
  --output_dir=$OUTPUT_DIR

In [0]:
!python transformers-master/examples/run_squad.py  --model_type bert  --model_name_or_path biobert_v1.1_pubmed/   --do_train  --do_lower_case  \
  --train_file=Data/bert_train.json \
  --do_eval \
  --predict_file=Data/bert_dev.json \
   --per_gpu_train_batch_size=24 \
  --learning_rate=3e-5 \
  --num_train_epochs=2.0 \
  --max_seq_length=384 \
  --doc_stride=128 \
  --logging_steps 15000 --save_steps 15000 \
  --output_dir=$OUTPUT_DIR

In [0]:
import json
import random
from nltk.corpus import stopwords
from nltk.cluster.util import cosine_distance
import numpy as np

class ContextSummary():
    def __init__(self, passage, question, top_n=12):
        stop_words = stopwords.words('english')
        summarize_text = []
    
        # Step 1 - Read text and split it
        sentences = passage.split(" .")
        #print(len(sentences))
        
        # Step 2 - Generate Similary Martix across sentences
        sentence_similarity_martix = self.build_similarity_matrix(sentences, question, stop_words)
        #print(sentence_similarity_martix.shape)
        
        # Step 3 - Sort the rank and pick top sentences
        ranked_sentence = sorted(((sentence_similarity_martix[i],s) for i,s in enumerate(sentences)), reverse=True)    
        #print("Indexes of top ranked_sentence order are ", ranked_sentence)    
    
        for i in range(top_n):
          summarize_text.append("".join(ranked_sentence[i][1]))
        #print(len(summarize_text))
        
        # Step 4 - Offcourse, output the summarize texr
        self.summary = ". ".join(summarize_text)
        print("Summarize Text: \n" + self.summary)  
        #return summary
    
    def sentence_similarity(self, sent1, ques, stopwords=None):
        if stopwords is None:
            stopwords = []
    
        sent1 = [w.lower() for w in sent1]
        ques = [w.lower() for w in ques]
     
        all_words = list(set(sent1 + ques))
     
        vector1 = [0] * len(all_words)
        vector2 = [0] * len(all_words)
     
        # build the vector for the first sentence
        for w in sent1:
            if w in stopwords:
                continue
            vector1[all_words.index(w)] += 1
     
        # build the vector for the second sentence
        for w in ques:
            if w in stopwords:
                continue
            vector2[all_words.index(w)] += 1
     
        return 1 - cosine_distance(vector1, vector2)
     
    def build_similarity_matrix(self, sentences, question, stop_words):
        # Create an empty similarity matrix
        similarity_matrix = np.zeros((len(sentences)))
        for idx1 in range(len(sentences)):
            similarity_matrix[idx1] = self.sentence_similarity(sentences[idx1], question, stop_words)
    
        return similarity_matrix

In [0]:
file_path = 'dev1.0.json'
output = open('results.txt', 'w+')

with open(file_path) as f:
    data = json.load(f)

### FILE ###
N = len(data['data'])
print("Total = ", N)

#Picking n random passages
pass_n = 2
passages = []

for i in range(0, pass_n):
    temp1 = random.randint(0, N-1)
    passages.append(temp1)

print("Chosen Passages:", passages)

for passage in passages:
    print("Passage ", passage)
    print("Document Title:  ", data['data'][passage]['document']['title'])
    #print(type(data['data'][passage]['document']['context']))
    pass_str = str(data['data'][passage]['document']['context'])
    ques_n = len(data['data'][passage]['document']['qas'])
    print("Questions = ", ques_n)
    for question in range(0, ques_n):
        print("Q "+str(question)+": "+str(data['data'][passage]['document']['qas'][question]['query'])+"\n")
        ques_str = str(data['data'][passage]['document']['qas'][question]['query'])
        short_pass = ContextSummary(pass_str, ques_str)
        print("Before: ", len(pass_str), "\nAfter: ", len(short_pass.summary))
        print("\n\n")

In [0]:
from transformers import BertModel
model = BertModel.from_pretrained('biobert_v1.1_pubmed/')

In [0]:
!python CSE576/transformers-master/examples/run_squad.py  --model_type bert   --model_name_or_path CSE576/Data/model/ --do_train --do_eval  --do_lower_case --overwrite_output_dir --train_file CSE576/data/Output_updated/bert_train.json --predict_file CSE576/data/Output_updated/bert_dev.json  --per_gpu_train_batch_size 16 --learning_rate 3e-5 --num_train_epochs 2.0  --max_seq_length 128  --gradient_accumulation_steps 2 --doc_stride 32 --output_dir CSE576/Data/Output/output_biobert_clicr_seqlen128_700 --logging_steps 5000 --save_steps 5000 --overwrite_output_dir

In [0]:
! pip install pytorch_pretrained_bert

     |████████████████████████████████| 133kB 3.4MB/s 


In [0]:
!pytorch_pretrained_bert convert_tf_checkpoint_to_pytorch biobert_v1.1_pubmed/model.ckpt-1000000 biobert_v1.1_pubmed/bert_config.json biobert_v1.1_pubmed/pytorch_model.bin

Building PyTorch model from configuration: {
  "attention_probs_dropout_prob": 0.1,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "max_position_embeddings": 512,
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "type_vocab_size": 2,
  "vocab_size": 28996
}

Converting TensorFlow checkpoint from /gdrive/My Drive/biobert_v1.1_pubmed/model.ckpt-1000000
Loading TF weight bert/embeddings/LayerNorm/beta with shape [768]
Loading TF weight bert/embeddings/LayerNorm/gamma with shape [768]
Loading TF weight bert/embeddings/position_embeddings with shape [512, 768]
Loading TF weight bert/embeddings/token_type_embeddings with shape [2, 768]
Loading TF weight bert/embeddings/word_embeddings with shape [28996, 768]
Loading TF weight bert/encoder/layer_0/attention/output/LayerNorm/beta with shape [768]
Loading TF weight bert/encoder/layer_0/attention/output/LayerNorm/gamma with shape [768]
Loading TF we

In [0]:
predictions_file = 'Data/output_bertlarge_clicr_nbest_predictions.json'
import json

with open(predictions_file,"r") as read_file:
  data = json.load(read_file)
new_predictions = 'Data/new_predictions.json'
new_dict = {}

In [0]:
len(data.keys())

3558

In [0]:
for _data in data.keys():
    if(data[_data][0]['text'] == ''):
      new_dict[_data] = data[_data][1]['text']
      count+=1
    else:
      new_dict[_data] = data[_data][0]['text']


In [0]:
new_dict['bcr-2012-006996.4']
count

1864

In [0]:
with open(new_predictions, 'w') as write_file:
  json.dump(new_dict, write_file)

In [4]:
! python Data/eval2.0.py New-Output-IV/bert_test.json Data/output_biobertbase_pubmed_pmc_2.0_test_predictions.json

{
  "exact": 52.19933184855234,
  "f1": 53.037111753008176,
  "total": 7184,
  "HasAns_exact": 52.19933184855234,
  "HasAns_f1": 53.037111753008176,
  "HasAns_total": 7184
}


In [0]:
import re
import json
import datetime
import os
import pprint
import random
import string

def normalize_answer(s, lemmatizer_comm=None):
    """Lower text and remove punctuation, articles and extra whitespace."""
    def remove_articles(text):
        return re.sub(r'\b(a|an|the)\b', ' ', text)

    def white_space_fix(text):
        return ' '.join(text.split())

    def remove_punc(text):
        exclude = set(string.punctuation)
        return ''.join(ch for ch in text if ch not in exclude)

    def lower(text):
        return text.lower()

    def lemma(text):
        if lemmatizer_comm is None:
            lemmatized = text
        else:
            lemmatized = lemmatize(text, lemmatizer_comm)
        return lemmatized

    return white_space_fix(remove_articles(remove_punc(lower(lemma(s)))))

In [0]:
def load_json(filename):
    with open(filename) as in_f:
        return json.load(in_f)

DATA_KEY = "data"
VERSION_KEY = "version"
DOC_KEY = "document"
QAS_KEY = "qas"
ANS_KEY = "answers"
TXT_KEY = "text"  # the text part of the answer
ORIG_KEY = "origin"
ID_KEY = "id"
TITLE_KEY = "title"
CONTEXT_KEY = "context"
SOURCE_KEY = "source"
QUERY_KEY = "query"
CUI_KEY = "cui"
SEMTYPE_KEY = "sem_type"
ISIMPOSSIBLE_KEY = "is_impossible"
PLACEHOLDER_KEY = "@placeholder"

In [0]:
def f1_score(prediction, ground_truth, comm=None):
    prediction_tokens = normalize_answer(prediction, comm).split()
    ground_truth_tokens = normalize_answer(ground_truth, comm).split()
    common = Counter(prediction_tokens) & Counter(ground_truth_tokens)
    num_same = sum(common.values())
    if num_same == 0:
        return 0
    precision = 1.0 * num_same / len(prediction_tokens)
    recall = 1.0 * num_same / len(ground_truth_tokens)
    f1 = (2 * precision * recall) / (precision + recall)
    return f1


def exact_match_score(prediction, ground_truth, comm=None):
    return normalize_answer(prediction, comm) == normalize_answer(ground_truth, comm)


def metric_max_over_ground_truths(metric_fn, prediction, ground_truths, comm=None):
    scores_for_ground_truths = []
    for ground_truth in ground_truths:
        score = metric_fn(prediction, ground_truth, comm)
        scores_for_ground_truths.append(score)
    return max(scores_for_ground_truths)

In [0]:
from collections import  Counter

def evaluate(dataset, predictions, lemmatizer_path=None, extended=False, embeddings_file=None, downcase=False):
    data = dataset[DATA_KEY]
    comm = lemmatizer(lemmatizer_path) if lemmatizer_path else None
    f1 = exact_match = total = 0

    n_unanswered = 0
    datum_count = 0
    
    for datum in data:
      for qa in datum[DOC_KEY][QAS_KEY]:
          ground_truths = []
          total += 1
          if qa[ID_KEY] not in predictions:
              n_unanswered += 1
              continue    
          ground_truths = list(map(lambda x: x[TXT_KEY], qa[ANS_KEY]))
          prediction = predictions[qa[ID_KEY]]
          exact_match += metric_max_over_ground_truths(
              exact_match_score, prediction, ground_truths, comm=comm)
          f1 += metric_max_over_ground_truths(
              f1_score, prediction, ground_truths, comm=comm)
      datum_count += 1
    
    print("There were {} unanswered instances".format(n_unanswered))
    exact_match = 100.0 * exact_match / total
    print(total)
    f1 = 100.0 * f1 / total
    # assert exact_match <= f1
    scores = {'exact_match': exact_match, 'f1': f1}
    return scores

In [0]:
def print_scores(scores):
    """
    :param scores: {"method1": score, ...}
    """
    print("{}\t{:.1f}".format("exact_match", scores["exact_match"]))
    print("{}\t{:.1f}".format("f1", scores["f1"]))

    for method, score in sorted(scores.items()):
        if method == "exact_match" or method == "f1":
            continue
        else:
            print("{}\t{:.3f}".format(method, score))

In [7]:
dataset = load_json('New-Output-III/bert_dev.json')
predictions = load_json('Data/output_biobertlarge_summarized_1.0_predictions.json')
print_scores(evaluate(dataset, predictions))

There were 1048 unanswered instances
3558
exact_match	30.0
f1	31.8


In [5]:
! pip install jsonlines

In [0]:
import jsonlines
import sys

file = 'New-Output-II/bert_dev.json'

In [11]:
a = 'apple'
b = 'frog'
a != b

True

In [0]:
gold_answers = {}

for _doc in range(0,len(data['data'])):
  document = data['data'][_doc]['document']
  qas = document['qas']
  
  for _qa in qas:
    gold_answers[_qa['id']] = _qa['answers']['text']

In [0]:
with open("New-Output-II/dev_answers.json",'w') as outfile:    
  json.dump(gold_answers,outfile)

In [0]:
data = load_json(file)

In [0]:
true_answers = load_json('New-Output-II/dev_answers.json')
predictions =  load_json('Data/output_bertlarge_clicr_2.0_predictions.json')

In [0]:
count = 0

In [16]:
for ques in true_answers:
  if(true_answers[ques] != '' and predictions[ques] != '' and (true_answers[ques] != predictions[ques])):
    print(ques+"         "+true_answers[ques]+"            "+predictions[ques])
    count+=1
count

bcr-2013-010177.3         pleural effusion            large pleural effusions
bcr-2013-203318.5         acute myeloid leukaemia M3            APL
bcr-2013-201462.1         Cardiac MRI            Delayed enhancement
bcr.08.2011.4575.6         performing adrenalectomy            unilateral tumours
bcr-2016-215445.3         cryptogenic organising pneumonia            cryptogenic OP
bcr-2016-215445.4         idiopathic interstitial pneumonias            major IIPs
bcr-2016-214994.3         CLE            Congenital lobar emphysema
bcr-2015-213819.3         unexplained left ventricular hypertrophy            left ventricular hypertrophy
bcr-2015-213819.6         cardiomyopathy            Fabry cardiomyopathy
bcr-2013-009979.2         myocardial infarction            acute myocardial infarction
bcr-2014-205723.9         small millimetric hypodensities along the cerebral sulci            small millimetric hypodensities
bcr-2014-205723.12         angiographic anatomy            angiographic an

248

In [0]:
len(list(predictions))

6391

In [0]:
true_answers

{'bcr-2012-006996.1': 'Caffey disease',
 'bcr-2012-006996.2': 'infantile cortical hyperostosis',
 'bcr-2012-006996.3': '',
 'bcr-2012-006996.4': 'systemic symptoms',
 'bcr-2012-006996.5': 'irritability',
 'bcr-2012-006996.6': 'fever',
 'bcr-2012-006996.7': '',
 'bcr-2012-006996.8': 'underlying cortical bone thickening',
 'bcr-2012-006996.10': '',
 'bcr-2012-006996.11': '',
 'bcr-2012-006996.12': '',
 'bcr-2012-006996.13': 'Caffey disease',
 'bcr-2012-006996.14': '',
 'bcr-2012-006996.15': '',
 'bcr-2012-006996.16': '',
 'bcr-2012-006996.17': '',
 'bcr-2012-006996.18': '',
 'bcr-2013-010177.1': 'expansion pulmonary edema',
 'bcr-2013-010177.2': 'pneumothorax',
 'bcr-2013-010177.3': 'pleural effusion',
 'bcr-2013-010177.4': 'thoracentesis',
 'bcr-2013-010177.5': 'chest tightness',
 'bcr-2013-010177.6': '',
 'bcr-2013-010177.7': 'procedure',
 'bcr-2012-006642.1': 'skin lesion',
 'bcr-2012-006642.2': 'skin lesion in axilla',
 'bcr-2012-006642.3': '',
 'bcr-2013-008788.1': 'constrictive per

In [0]:
predictions

{'bcr-2012-006996.1': '',
 'bcr-2012-006996.2': '',
 'bcr-2012-006996.3': '',
 'bcr-2012-006996.4': '',
 'bcr-2012-006996.5': '',
 'bcr-2012-006996.6': '',
 'bcr-2012-006996.7': '',
 'bcr-2012-006996.8': '',
 'bcr-2012-006996.10': '',
 'bcr-2012-006996.11': '',
 'bcr-2012-006996.12': '',
 'bcr-2012-006996.13': 'Caffey disease',
 'bcr-2012-006996.14': 'childhood bone swelling',
 'bcr-2012-006996.15': '',
 'bcr-2012-006996.16': '',
 'bcr-2012-006996.17': '',
 'bcr-2012-006996.18': '',
 'bcr-2013-010177.1': 'expansion pulmonary edema',
 'bcr-2013-010177.2': 'pneumothorax',
 'bcr-2013-010177.3': 'large pleural effusions',
 'bcr-2013-010177.4': '',
 'bcr-2013-010177.5': '',
 'bcr-2013-010177.6': 'experiences coughing',
 'bcr-2013-010177.7': '',
 'bcr-2012-006642.1': 'skin lesion',
 'bcr-2012-006642.2': '',
 'bcr-2012-006642.3': '',
 'bcr-2013-008788.1': 'constrictive pericarditis',
 'bcr-2013-008788.2': '',
 'bcr-2013-008788.3': '',
 'bcr-2013-008788.4': '',
 'bcr-2013-008788.5': '',
 'bcr-

In [0]:
empty_true_answers = []
non_empty_true_answers = []

for _ans in true_answers.keys():
  if true_answers[_ans] == '':
    empty_true_answers.append(_ans)
  else:
    non_empty_true_answers.append(_ans)
    
len(non_empty_true_answers)

3558

In [0]:
empty_predictions = []
non_empty_predictions = []

for _ans in predictions.keys():
  if predictions[_ans] == '':
    empty_predictions.append(_ans)
  else:
    non_empty_predictions.append(_ans)


In [0]:
empty_empty = set(empty_true_answers).intersection(set(empty_predictions))
empty_nonempty = set(empty_true_answers).intersection(set(non_empty_predictions))
nonempty_empty = set(non_empty_true_answers).intersection(set(empty_predictions))
nonempty_nonempty = set(non_empty_true_answers).intersection(set(non_empty_predictions))

In [0]:
counts = {}
counts['empty_empty'] = len(empty_empty)
counts['empty_nonempty'] = len(empty_nonempty)
counts['nonempty_empty'] = len(nonempty_empty)
counts['nonempty_nonempty'] = len(nonempty_nonempty)
counts['gp<ap'] = 322

In [0]:
counts

{'empty_empty': 2662,
 'empty_nonempty': 171,
 'gp<ap': 322,
 'nonempty_empty': 2644,
 'nonempty_nonempty': 914}

In [0]:
len(non_empty_predictions)

1085

In [0]:
len(empty_predictions)

5306

In [0]:
count = 0
for key in predictions.keys():
  if ( len(true_answers[key]) < len(predictions[key])):
    print(key)
    count+=1
count

bcr-2012-006996.14
bcr-2013-010177.3
bcr-2013-010177.6
bcr-2014-206913.1
bcr-2013-201462.1
bcr-2016-214994.3
bcr.12.2011.5307.4
bcr-2015-213819.6
bcr-2015-209925.1
bcr-2015-209925.5
bcr-2013-009979.2
bcr-2014-205723.12
bcr-2013-202196.2
bcr-2014-204928.2
bcr-2013-202212.7
bcr.05.2010.3039.1
bcr.05.2010.3039.2
bcr.06.2011.4304.13
bcr.06.2011.4304.15
bcr.11.2010.3539.7
bcr.10.2008.1163.1
bcr-2012-007756.3
bcr-2012-007756.4
bcr-2014-204631.2
bcr-2014-204631.4
bcr.04.2010.2896.12
bcr-2014-205100.4
bcr.03.2009.1645.18
bcr.05.2011.4275.3
bcr.10.2011.4935.3
bcr.10.2011.4935.5
bcr.10.2011.4935.7
bcr.10.2011.4935.9
bcr-2013-009111.1
bcr-2013-009111.3
bcr-2015-213333.2
bcr-2013-201742.3
bcr-2014-206475.12
bcr-2014-207778.5
bcr-2014-206609.6
bcr.06.2011.4309.2
bcr.06.2011.4309.4
bcr-2014-207147.1
bcr-2014-207147.9
bcr-2014-209153.3
bcr-2014-207420.2
bcr-2014-207420.7
bcr-2016-215702.10
bcr-2016-215702.15
bcr-2016-215702.16
bcr-2016-214721.8
bcr.11.2009.2486.4
bcr.11.2008.1179.11
bcr-2016-214626.3

322

In [0]:
em_em = list(empty_empty)[0]
em_nem = list(empty_nonempty)[0]
nem_em = list(nonempty_empty)[0]
nem_nem = list(nonempty_nonempty)[0]

In [0]:
samples = {}
samples['em_em'] = em_em
samples['em_nem'] = em_nem
samples['nem_em'] = nem_em
samples['nem_nem'] = nem_nem
samples['gp<ap'] = 'bcr-2013-010177.3'

In [0]:
samples

{'em_em': 'bcr-2014-207995.2',
 'em_nem': 'bcr-2015-214104.5',
 'gp<ap': 'bcr-2013-010177.3',
 'nem_em': 'bcr-2013-200212.1',
 'nem_nem': 'bcr.02.2011.3890.25'}

In [0]:
for _doc in range(0,len(data['data'])):
  document = data['data'][_doc]['document']
  qas = document['qas']

  for _qa in qas:
    if(_qa['id'] == samples['gp<ap']):
      print(_qa)
      print(document['context'])
      

{'answers': {'text': 'pleural effusion', 'origin': 'dataset', 'sem_type': 'problem', 'cui': 'C0032227', 'answer_start': 167}, 'id': 'bcr-2013-010177.3', 'query': 'Re- expansion pulmonary edema is a rare but potentially fatal condition which can occur while managing patients with pneumothorax or  .', 'is_impossible': False}
Summary

Re- expansion pulmonary edema ( RPE ) is a rare but potentially fatal complication that can occur following rapid lung expansion while managing patients with pleural effusion or pneumothorax .
In this case , fatal outcome occurred due to RPE in a previously healthy young adult male patient subsequent to tube thoracostomy for spontaneous pneumothorax .
While managing patients with pneumothorax or large pleural effusions , precautions should be taken to avoid rapid re-expansion of the previously collapsed lung in order to reduce the probability of development of this complication .

Background

Re- expansion pulmonary edema ( RPE ) is a rare complication which

In [0]:
print(predictions[samples['gp<ap']])

large pleural effusions


In [4]:
! pip install pandas
! pip install jsonlines

In [0]:
import json
import jsonlines
import pandas as pd

In [0]:
file = "Data/train1.0.json"
with jsonlines.open(file, 'r') as f:
    data = f.read()

In [0]:
contraction_mapping = {"ain't": "is not", "aren't": "are not","can't": "cannot", "'cause": "because", "could've": "could have", "couldn't": "could not",

                           "didn't": "did not", "doesn't": "does not", "don't": "do not", "hadn't": "had not", "hasn't": "has not", "haven't": "have not",

                           "he'd": "he would","he'll": "he will", "he's": "he is", "how'd": "how did", "how'd'y": "how do you", "how'll": "how will", "how's": "how is",

                           "I'd": "I would", "I'd've": "I would have", "I'll": "I will", "I'll've": "I will have","I'm": "I am", "I've": "I have", "i'd": "i would",

                           "i'd've": "i would have", "i'll": "i will",  "i'll've": "i will have","i'm": "i am", "i've": "i have", "isn't": "is not", "it'd": "it would",

                           "it'd've": "it would have", "it'll": "it will", "it'll've": "it will have","it's": "it is", "let's": "let us", "ma'am": "madam",

                           "mayn't": "may not", "might've": "might have","mightn't": "might not","mightn't've": "might not have", "must've": "must have",

                           "mustn't": "must not", "mustn't've": "must not have", "needn't": "need not", "needn't've": "need not have","o'clock": "of the clock",

                           "oughtn't": "ought not", "oughtn't've": "ought not have", "shan't": "shall not", "sha'n't": "shall not", "shan't've": "shall not have",

                           "she'd": "she would", "she'd've": "she would have", "she'll": "she will", "she'll've": "she will have", "she's": "she is",

                           "should've": "should have", "shouldn't": "should not", "shouldn't've": "should not have", "so've": "so have","so's": "so as",

                           "this's": "this is","that'd": "that would", "that'd've": "that would have", "that's": "that is", "there'd": "there would",

                           "there'd've": "there would have", "there's": "there is", "here's": "here is","they'd": "they would", "they'd've": "they would have",

                           "they'll": "they will", "they'll've": "they will have", "they're": "they are", "they've": "they have", "to've": "to have",

                           "wasn't": "was not", "we'd": "we would", "we'd've": "we would have", "we'll": "we will", "we'll've": "we will have", "we're": "we are",

                           "we've": "we have", "weren't": "were not", "what'll": "what will", "what'll've": "what will have", "what're": "what are",

                           "what's": "what is", "what've": "what have", "when's": "when is", "when've": "when have", "where'd": "where did", "where's": "where is",

                           "where've": "where have", "who'll": "who will", "who'll've": "who will have", "who's": "who is", "who've": "who have",

                           "why's": "why is", "why've": "why have", "will've": "will have", "won't": "will not", "won't've": "will not have",

                           "would've": "would have", "wouldn't": "would not", "wouldn't've": "would not have", "y'all": "you all",

                           "y'all'd": "you all would","y'all'd've": "you all would have","y'all're": "you all are","y'all've": "you all have",

                           "you'd": "you would", "you'd've": "you would have", "you'll": "you will", "you'll've": "you will have",

                           "you're": "you are", "you've": "you have"}

In [21]:
! pip install nltk

In [0]:
import numpy as np  
import pandas as pd 
import re           
from bs4 import BeautifulSoup 
from keras.preprocessing.text import Tokenizer 
from keras.preprocessing.sequence import pad_sequences
from nltk.corpus import stopwords   
from tensorflow.keras.layers import Input, LSTM, Embedding, Dense, Concatenate, TimeDistributed, Bidirectional
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping
import warnings

In [24]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [0]:
stop_words = set(stopwords.words('english')) 
def text_cleaner(text):
    newString = text.lower()
    newString = BeautifulSoup(newString, "lxml").text
    newString = re.sub(r'\([^)]*\)', '', newString)
    newString = re.sub('"','', newString)
    newString = ' '.join([contraction_mapping[t] if t in contraction_mapping else t for t in newString.split(" ")])    
    newString = re.sub(r"'s\b","",newString)
    newString = re.sub("[^a-zA-Z]", " ", newString) 
    tokens = [w for w in newString.split() if not w in stop_words]
    long_words=[]
    for i in tokens:
        if len(i)>=3:                  #removing short word
            long_words.append(i)   
    return (" ".join(long_words)).strip()

In [56]:
_data = data['data']
_data[0]['document'].keys()

dict_keys(['qas', 'title', 'context'])

In [0]:
a= []
context= pd.DataFrame([])

for i in range(0,len(data['data'])):
    b=data['data'][i]['document']['context']
    a.append(b)
    
df = pd.DataFrame(a, columns=['context'])

In [58]:
s=df.iloc[1]
print ("the length of the original text:",len(s[0]))
print ("the length of the summerized one is:",len(cleaned_text[1]))

s=df.iloc[2]
print ("the length of the original text:",len(s[0]))
print ("the length of the summerized one is:",len(cleaned_text[2]))

s=df.iloc[3]
print ("the length of the original text:",len(s[0]))
print ("the length of the summerized one is:",len(cleaned_text[3]))

the length of the original text: 7990


NameError: ignored

In [0]:
for _context in range(0,df.shape[0]):
  data['data'][_context]['document']['context'] = text_cleaner(df.iloc[_context]['context'])


In [45]:
df.iloc[0]['context']

"Summary\n\nBEG__Ocular adnexal lymphomas__END account for 1 – 2 % of all BEG__non-Hodgkin ' s lymphomas__END .\nConjunctiva is the primary site of involvement in one - third of cases .\nWe present a case of a 47 - year - old Hispanic woman who presented with BEG__left eye itching__END and BEG__irritation__END associated with a BEG__painless pink mass__END .\nBEG__Physical examination__END revealed the presence of a ‘ pink salmon - patch ’ involving her left medial conjunctiva .\nBEG__Orbital CT__END showed a BEG__subcentimeter left preseptal soft tissue density__END .\nBEG__Biopsy__END revealed a BEG__dense subepithelial lymphoid infiltrate__END comprised predominantly of B cells that did not coexpress CD5 or CD43 .\nThese BEG__findings__END were consistent with BEG__B - cell marginal zone lymphoma__END .\nBEG__Further staging assessment__END did not reveal BEG__disseminated disease__END .\nShe had BEG__stage 1E extranodal marginal zone lymphoma__END as per Ann Arbor staging system .\

In [0]:
with open("New-Output-V/train1.0.json",'w') as outfile:    # writes the converted data into the file
  json.dump(data,outfile)

In [0]:
file = "Data/train1.0.json"
with jsonlines.open(file, 'r') as f:
    _data = f.read()

In [0]:
file = "New-Output-V/train1.0.json"
with jsonlines.open(file, 'r') as f:
    data = f.read()

In [67]:
for _doc in data['data']:
  _context = _doc['document']['context']
  print(len(_context))

Streaming output truncated to the last 5000 lines.
3360
7449
2061
8079
5651
4003
10458
10410
13945
17491
11317
3416
24261
7995
10270
8003
8411
9012
6613
9262
3491
9733
1129
12210
9221
9113
7710
1742
8232
4753
12860
9000
13764
15126
2597
11810
7181
770
12542
12341
12415
10070
2947
11482
11892
13475
12290
36234
9118
10496
3226
7519
11407
2597
4844
12368
10824
23670
8079
13169
2391
11740
15312
9928
6618
13075
6699
9697
9780
9598
10531
2250
15356
9712
2933
2676
12034
8769
5842
26138
5560
10884
14443
9155
15161
21083
11424
10564
2242
9433
14701
12743
9709
8977
5637
14696
7089
13330
9947
13913
11260
5565
11684
1517
12274
13695
9218
2326
10349
16109
5947
8516
10530
15017
13610
10825
18677
8767
2640
11737
3254
23077
16313
6745
9972
3633
12443
14323
10383
12858
17333
1954
3264
2524
9044
6972
10796
6505
12678
9161
11800
9750
2121
12268
2628
6592
5652
2598
3004
2814
8066
3173
14539
11368
5422
8567
5366
11322
14721
5657
14110
7020
10202
10076
14793
11138
3041
18852
6320
10369
15784
6716
2144
9859


In [68]:
for _doc in _data['data']:
  _context = _doc['document']['context']
  print(len(_context))
  

Streaming output truncated to the last 5000 lines.
3360
7449
2061
8079
5651
4003
10458
10410
13945
17491
11317
3416
24261
7995
10270
8003
8411
9012
6613
9262
3491
9733
1129
12210
9221
9113
7710
1742
8232
4753
12860
9000
13764
15126
2597
11810
7181
770
12542
12341
12415
10070
2947
11482
11892
13475
12290
36234
9118
10496
3226
7519
11407
2597
4844
12368
10824
23670
8079
13169
2391
11740
15312
9928
6618
13075
6699
9697
9780
9598
10531
2250
15356
9712
2933
2676
12034
8769
5842
26138
5560
10884
14443
9155
15161
21083
11424
10564
2242
9433
14701
12743
9709
8977
5637
14696
7089
13330
9947
13913
11260
5565
11684
1517
12274
13695
9218
2326
10349
16109
5947
8516
10530
15017
13610
10825
18677
8767
2640
11737
3254
23077
16313
6745
9972
3633
12443
14323
10383
12858
17333
1954
3264
2524
9044
6972
10796
6505
12678
9161
11800
9750
2121
12268
2628
6592
5652
2598
3004
2814
8066
3173
14539
11368
5422
8567
5366
11322
14721
5657
14110
7020
10202
10076
14793
11138
3041
18852
6320
10369
15784
6716
2144
9859
